In [1]:
#########
# IMPORTS
#########
import random as rand
import tensorflow as tf
import networkx as nx
import numpy as np
import pandas as pd
import time
import pickle
from tsp_solver.greedy import solve_tsp as solve
from graph_nets import utils_tf
from graph_nets import utils_np
from graph_nets.demos import models


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
#########
# UTILS
#########


def create_random_graph(node_range=(5, 9), prob=0.25, weight_range=(1, 10)):
    n_nodes = rand.randint(*node_range)

    G = nx.complete_graph(n_nodes)
    H = G.copy()
    for u, v, w in G.edges(data=True):
        H[u][v]["weight"] = rand.randint(*weight_range)

        # u_deg, v_deg = H.degree(u), H.degree(v)
        # if u_deg - 1 >= n_nodes / 2 and v_deg - 1 >= n_nodes / 2:
        #     if rand.random() < prob:
        #         H.remove_edge(u, v)

    return H


def solve_tsp(graph):
    adj_matrix = nx.adjacency_matrix(graph)
    hamil_path = solve(adj_matrix.todense().tolist())

    path_edges = [(hamil_path[i], hamil_path[i + 1])
                  for i in range(len(hamil_path) - 1)]
    path_edges.append((hamil_path[-1], hamil_path[0]))

    for u, v in graph.edges():
        graph[u][v]["solution"] = int(
            any({u, v}.issubset({src, targ}) for src, targ in path_edges))

    solution_dict = {v: False for v in graph.nodes()}
    for u, v in path_edges:
        solution_dict[u] = True
        solution_dict[v] = True

    nx.set_node_attributes(graph, solution_dict, "solution")

    return graph


def to_one_hot(indices, max_value, axis=-1):
    one_hot = np.eye(max_value)[indices]
    if axis not in (-1, one_hot.ndim):
        one_hot = np.moveaxis(one_hot, -1, axis)

    return one_hot


def graph_to_input_target(graph):
    """Returns 2 graphs with input and target feature vectors for training.
    Args:
    graph: An `nx.Graph` instance.
    Returns:
    The input `nx.Graph` instance.
    The target `nx.Graph` instance.
    Raises:
    ValueError: unknown node type
    """

    def create_feature(attr, fields):
        return np.hstack([np.array(attr[field], dtype=float) for field in fields])

    input_node_fields = ("solution",)
    input_edge_fields = ("weight",)
    target_node_fields = ("solution",)
    target_edge_fields = ("solution",)

    input_graph = graph.copy()
    target_graph = graph.copy()

    solution_length = 0
    for node_index, node_feature in graph.nodes(data=True):
        input_graph.add_node(
            node_index, features=create_feature(node_feature, input_node_fields))
        target_node = to_one_hot(
            create_feature(node_feature, target_node_fields).astype(int), 2)[0]
        target_graph.add_node(node_index, features=target_node)

    for receiver, sender, features in graph.edges(data=True):
        input_graph.add_edge(
            sender, receiver, features=create_feature(features, input_edge_fields))
        target_edge = to_one_hot(
            create_feature(features, target_edge_fields).astype(int), 2)[0]
        target_graph.add_edge(sender, receiver, features=target_edge)
        solution_length += features["weight"] * features["solution"]

    input_graph.graph["features"] = np.array([0.0])
    target_graph.graph["features"] = np.array([solution_length], dtype=float)

    # print(type(input_graph))
    # print(input_graph.graph)
    # print(type(target_graph))
    # print(target_graph.graph)

    return input_graph, target_graph


def generate_networkx_graphs(num_graphs, node_range=(5, 9), prob=0.25, weight_range=(1, 10)):
    """Generate graphs for training.
    Args:
    num_graphs: number of graphs to generate
    num_range: a 2-tuple with the [lower, upper) number of nodes per
      graph
    prob: the probability of removing an edge between any two nodes
    weight_range: a 2-tuple with the [lower, upper) weight to randomly assign
        to (non-removed) edges
    Returns:
    input_graphs: The list of input graphs.
    target_graphs: The list of output graphs.
    graphs: The list of generated graphs.
    """

    input_graphs = []
    target_graphs = []
    graphs = []

    for i in range(num_graphs):
        graph = create_random_graph(node_range, prob, weight_range)
        graph = solve_tsp(graph)
        input_graph, target_graph = graph_to_input_target(graph)
        input_graphs.append(input_graph)
        target_graphs.append(target_graph)
        graphs.append(graph)

    return input_graphs, target_graphs, graphs


#########
# EXPORTS
#########


def create_placeholders(num_graphs):
    input_graphs, target_graphs, _ = generate_networkx_graphs(num_graphs)
    input_ph = utils_tf.placeholders_from_networkxs(input_graphs)
    target_ph = utils_tf.placeholders_from_networkxs(target_graphs)
    return input_ph, target_ph


def make_all_runnable_in_session(*args):
    """Lets an iterable of TF graphs be output from a session as NP graphs."""
    return [utils_tf.make_runnable_in_session(a) for a in args]


def create_feed_dict(num_graphs, input_ph, target_ph):
    """Creates placeholders for the model training and evaluation."""

    inputs, targets, raw_graphs = generate_networkx_graphs(num_graphs)
    input_graphs = utils_np.networkxs_to_graphs_tuple(inputs)
    target_graphs = utils_np.networkxs_to_graphs_tuple(targets)
    feed_dict = {input_ph: input_graphs, target_ph: target_graphs}

    return feed_dict, inputs


In [3]:
#########
# TRAINING_HELPER (THAT CAN BE TUNED)
#########
def create_loss_ops(target_op, output_ops):
    return [
        tf.losses.softmax_cross_entropy(target_op.edges, output_op.edges)
        for output_op in output_ops
    ]


def compute_accuracy(target, output, use_nodes=True, use_edges=False):
    if not use_nodes and not use_edges:
        raise ValueError("Nodes or edges (or both) must be used")

    tdds = utils_np.graphs_tuple_to_data_dicts(target)
    odds = utils_np.graphs_tuple_to_data_dicts(output)

    cs, ss = [], []
    for td, od in zip(tdds, odds):

        xe = np.argmax(td["edges"], axis=-1)
        ye = np.argmax(od["edges"], axis=-1)

        c = [xe == ye] if use_edges else []
        c = np.concatenate(c, axis=0)

        s = np.all(c)
        cs.append(c)
        ss.append(s)

    correct = np.mean(np.concatenate(cs, axis=0))
    solved = np.mean(np.stack(ss))

    return correct, solved

In [4]:
#########
# MODEL AND HYPERPARAMETERS SETUP
#########
tf.reset_default_graph()

seed = 2
rand = np.random.RandomState(seed=seed)

# Model parameters; no. of message-passing steps
num_processing_steps_tr = 10
num_processing_steps_ge = 10

# Data / training parameters
num_training_iterations = 1000
batch_size_tr = 100
batch_size_ge = 32

# Input and target placeholders
input_ph, target_ph = create_placeholders(batch_size_tr)

# Connect the data to the model and instantiate
model = models.EncodeProcessDecode(edge_output_size=2, node_output_size=2)
# A list of outputs, one per processing step
output_ops_tr = model(input_ph, num_processing_steps_tr)
output_ops_ge = model(input_ph, num_processing_steps_ge)

# Training loss
loss_ops_tr = create_loss_ops(target_ph, output_ops_tr)
# Loss across processing steps
loss_op_tr = sum(loss_ops_tr) / num_processing_steps_tr
# Test/generalization loss
loss_ops_ge = create_loss_ops(target_ph, output_ops_ge)
loss_op_ge = loss_ops_ge[-1]  # Loss from final processing step

# Optimizer
learning_rate = 1.3e-3
optimizer = tf.train.AdamOptimizer(learning_rate)
step_op = optimizer.minimize(loss_op_tr)

# Lets an iterable of TF graphs be output from a session as NP graphs
input_ph, target_ph = make_all_runnable_in_session(input_ph, target_ph)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


/Users/roger/Library/Python/3.7/lib/python/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [5]:
#########
# RESET SESSION
#########
try:
    sess.close()
except NameError:
    pass

sess = tf.Session()
sess.run(tf.global_variables_initializer())

last_iteration = 0
logged_iterations = []
losses_tr = []
corrects_tr = []
solveds_tr = []
losses_ge = []
corrects_ge = []
solveds_ge = []

In [6]:
#########
# TRAINING
#########
# How much time between logging and printing the current results.
log_every_seconds = 20

print("# (iteration number), T (elapsed seconds), "
      "Ltr (training loss), Lge (test/generalization loss), "
      "Ctr (training fraction nodes/edges labeled correctly), "
      "Str (training fraction examples solved correctly), "
      "Cge (test/generalization fraction nodes/edges labeled correctly), "
      "Sge (test/generalization fraction examples solved correctly)")

start_time = time.time()
last_log_time = start_time
for iteration in range(last_iteration, num_training_iterations):
    last_iteration = iteration

    feed_dict, _ = create_feed_dict(batch_size_tr, input_ph, target_ph)
    train_values = sess.run({
        "step": step_op,
        "target": target_ph,
        "loss": loss_op_tr,
        "outputs": output_ops_tr
    }, feed_dict=feed_dict)

    the_time = time.time()
    elapsed_since_last_log = the_time - last_log_time

    if elapsed_since_last_log > log_every_seconds:
        last_log_time = the_time

        feed_dict, raw_graphs = create_feed_dict(
            batch_size_ge, input_ph, target_ph)
        test_values = sess.run({
            "target": target_ph,
            "loss": loss_op_ge,
            "outputs": output_ops_ge
        }, feed_dict=feed_dict)

        correct_tr, solved_tr = compute_accuracy(
            train_values["target"], train_values["outputs"][-1], use_edges=True)
        correct_ge, solved_ge = compute_accuracy(
            test_values["target"], test_values["outputs"][-1], use_edges=True)

        elapsed = time.time() - start_time

        losses_tr.append(train_values["loss"])
        corrects_tr.append(correct_tr)
        solveds_tr.append(solved_tr)
        losses_ge.append(test_values["loss"])
        corrects_ge.append(correct_ge)
        solveds_ge.append(solved_ge)
        logged_iterations.append(iteration)

        print("# {:05d}, T {:.1f}, Ltr {:.4f}, Lge {:.4f}, Ctr {:.4f}, Str"
              " {:.4f}, Cge {:.4f}, Sge {:.4f}".format(
                  iteration, elapsed, train_values["loss"], test_values["loss"],
                  correct_tr, solved_tr, correct_ge, solved_ge))

# (iteration number), T (elapsed seconds), Ltr (training loss), Lge (test/generalization loss), Ctr (training fraction nodes/edges labeled correctly), Str (training fraction examples solved correctly), Cge (test/generalization fraction nodes/edges labeled correctly), Sge (test/generalization fraction examples solved correctly)
# 00055, T 21.1, Ltr 0.6438, Lge 0.6453, Ctr 0.6318, Str 0.0000, Cge 0.6264, Sge 0.0000
# 00137, T 40.4, Ltr 0.4320, Lge 0.4373, Ctr 0.8099, Str 0.0000, Cge 0.8183, Sge 0.0312
# 00223, T 60.5, Ltr 0.4202, Lge 0.4645, Ctr 0.8226, Str 0.0100, Cge 0.7927, Sge 0.0312
# 00308, T 80.6, Ltr 0.4477, Lge 0.4377, Ctr 0.8021, Str 0.0000, Cge 0.8017, Sge 0.0000
# 00394, T 100.7, Ltr 0.4174, Lge 0.3874, Ctr 0.8186, Str 0.0300, Cge 0.8356, Sge 0.0312
# 00480, T 120.9, Ltr 0.4220, Lge 0.3999, Ctr 0.8113, Str 0.0100, Cge 0.8177, Sge 0.0000
# 00565, T 141.0, Ltr 0.4138, Lge 0.4103, Ctr 0.8166, Str 0.0100, Cge 0.8124, Sge 0.0312
# 00651, T 161.0, Ltr 0.4209, Lge 0.4045, Ctr 0.8188

In [7]:
#########
# POST-TRAINING TESTING
#########
test_batch_size = 1000
num_processing_steps_test = 10

test_input_ph, test_target_ph = create_placeholders(test_batch_size)
test_output_ops = model(test_input_ph, num_processing_steps_test)

test_loss_ops = create_loss_ops(test_target_ph, test_output_ops)
test_loss_op = test_loss_ops[-1]

test_input_ph, test_target_ph = make_all_runnable_in_session(
    test_input_ph, test_target_ph)

test_feed_dict, test_input_graphs = create_feed_dict(
    test_batch_size, test_input_ph, test_target_ph)
test_values = sess.run({
    "target": test_target_ph,
    "loss": test_loss_op,
    "outputs": test_output_ops
}, feed_dict=test_feed_dict)

In [9]:
#########
# MODEL & RESULTS STORAGE
#########


# Store the model
saver = tf.train.Saver()
saver.save(sess, "data/pickles/model.ckpt")

# Store the training statistics
train_stats = pd.DataFrame(np.array([logged_iterations, losses_tr, losses_ge,
                                     corrects_tr, solveds_tr, corrects_ge, solveds_ge]).T,
                           columns=["iteration", "loss_tr", "loss_ge", "correct_tr", "solved_tr",
                                    "correct_ge", "solved_ge"])
train_stats.to_pickle("data/pickles/train_stats.pkl")

# Store the test results
pickle.dump({
    "outputs": utils_np.graphs_tuple_to_networkxs(test_values["outputs"][-1]),
    "targets": utils_np.graphs_tuple_to_networkxs(test_values["target"]),
    "inputs": test_input_graphs,
}, open("data/pickles/test_results.pkl", "wb"))